---
title: A Tutorial on Conducting Mediation Analysis with Exposure Mixtures
filters:
  - webr
---


## Load libraries required for this document

```{webr-r}
library(tidyverse)
# library(bkmr) bkmr is not available on webR
# library(CMAverse) CMAverse is not available on webR

# Can compile unavailable R packages to an R WASM Package binary and setup a custom repository to share it (r-universe.dev)
```


## Example: Simulation Data

### Setup code

```{webr-r}
p <- 1; q <- 30; s <- 6

# Indirect effects
Alpha_a <- matrix(0, nrow = 1, ncol = q)
Alpha_a[c(1, 11, 21)] <- 1 # weak effect
Alpha_a[c(2, 12, 22)] <- 4 # moderate effect
Alpha_a[c(3, 13, 23)] <- 8 # strong effect

Beta_m <- 0.5

# Direct effects
Beta_a <- rep(c(5, 0, 0), times = q/3) %>% as.matrix()

# Confounder effects
Theta_c <- matrix(rep(0.1, times = q*(s-1)), nrow = q)
Alpha_c <- matrix(1, nrow = p, ncol = s)
Beta_c <- matrix(1, nrow = s, ncol = 1)
```


### Data Generation

```{webr-r}
# Function that generates data
data_gen <- function(n_obs, n_expo, n_confound,
                     expo_blockNum, expo_blockCorr,
                     confound_blockNum, confound_blockCorr,
                     Alpha_a, Alpha_c, Beta_m, Beta_a, Beta_c,
                     Theta_c, # theta_c is what confounders contributes to exposures (q times s dim)
                     adjR2_M, adjR2_Y){
  n_obs <- n_obs
  q <- n_expo
  p <- 1 # we assume 1 mediator for now
  # We assume no confounders
  s <- n_confound + 1 # if s == 1 then it is just the intercept
  
  # Geenrate intercept and confounders
  if(s == 1){
    C_i_T <- rep(1, n_obs) %>% as.matrix() 
    Sigma_C <- diag(s)
  } else{
    interCept <- rep(1, n_obs) %>% as.matrix() 
    
    Sigma_C <- gen_block_corr(exposure_numbers = confound_blockNum,
                              correlations = confound_blockCorr)
    
    conFound <- MASS::mvrnorm(n = n_obs, mu = rep(0, s-1), Sigma = Sigma_C) # s is the number of confounders
    C_i_T <- cbind(interCept, conFound)
  }
  
  ## Exposures
  Sigma_X <- gen_block_corr(exposure_numbers = expo_blockNum,
                            correlations = expo_blockCorr)
  
  # generate the exposures
  X <- t(Theta_c %*% t(conFound)) + MASS::mvrnorm(n = n_obs, mu = rep(0, q), Sigma = Sigma_X) #  MASS::mvrnorm(n = n_obs, mu = rep(0, q), Sigma = Sigma_X)
  colnames(X) <- paste0("x", 1: q)
  
  # we can calculate the Sigma_M (a scalar here)
  adjR2_M <- adjR2_M
  r2_M <- 1 - ((n_obs - q - s - 1)/(n_obs - 1))*(1 - adjR2_M) # set adjusted r-squared to 0.3
  
  
  # some prep
  big_alpha <- cbind(Alpha_a, Alpha_c)
  colnames(big_alpha) <- NULL
  
  V_mat_alpha <- matrix(0, nrow = (q+s), ncol = (q+s))
  V_mat_alpha[1:q, 1:q] <- var(X)
  V_mat_alpha[(q+1):(q+s), (q+1):(q+s)] <- var(C_i_T)
  
  
  Sigma_M <- ((1-r2_M)/(r2_M))*big_alpha %*% V_mat_alpha %*% t(big_alpha)
  
  # Generate M Mediators
  M <- t(Alpha_a %*% t(X) + Alpha_c %*% t(C_i_T)) + MASS::mvrnorm(n = n_obs, mu = 0, Sigma = Sigma_M) # mu + error
  
  #Generate sigma^2_e (error variance of the outcome model)
  # big_beta
  big_bt <- cbind(Beta_m, t(Beta_a), t(Beta_c))
  colnames(big_bt) <- NULL
  
  # build the V = var covar of M, X
  V_mat <- matrix(0, nrow = (p+q+s), ncol = (p+q+s))
  V_mat[1:p, 1:p] <- var(M)
  V_mat[(p+1):(p+q), (p+1):(p+q)] <- var(X) 
  V_mat[(p+q+1):(p+q+s), (p+q+1):(p+q+s)] <- var(C_i_T) 
  
  
  # Calculate the sigma_y
  adjR2_Y <- adjR2_Y
  r2_Y <- 1 - ((n_obs - p - q - s - 1)/(n_obs - 1))*(1 - adjR2_Y) # set adjusted r-squared to 0.3
  
  # calculate the optimal sigma^2_e for the two cases
  Sigma_Y <- ((1-r2_Y)/(r2_Y))*big_bt %*% V_mat %*% t(big_bt)
  
  #Generate Y
  comb_predictors <- cbind(M, X, C_i_T)
  colnames(comb_predictors)[seq(p)] <- paste("m", seq(p))
  colnames(comb_predictors)[p + seq(q)] <- paste("x", seq(q))
  colnames(comb_predictors)[p + q + seq(s)] <- paste(c("intercept", paste("c", seq(s-1))))
  
  # if there are more than one confounders
  #ifelse(s > 1,   colnames(comb_predictors)[p + q + 1 + seq(s)] <- paste("c", seq(s-1)), 
  #      0)
  
  y1 <- t(big_bt %*% t(comb_predictors)) + MASS::mvrnorm(n = n_obs, mu = 0, Sigma = Sigma_Y) # mu + error
  
  #Combine the results
  df_gen <- cbind(y1, M, X, C_i_T) %>% as.data.frame() #%>% dplyr::rename(y = V1, m1 = V2, intercept = V33)
  
  colnames(df_gen) <- c("y", paste0("m", seq(p)), paste0("x", seq(q)), "intercept", paste0("c", seq(s-1)))
  
  return(df_gen)
}

gen_block_corr <- function(exposure_numbers, correlations) {
  if (length(exposure_numbers) != length(correlations)) {
    stop("The lengths of exposure_numbers and correlations must match.")
  }
  
  # Load the Matrix library for bdiag
  if (!requireNamespace("Matrix", quietly = TRUE)) {
    stop("The Matrix package is required. Please install it using install.packages('Matrix')")
  }
  
  # Function to create a single correlation block
  create_correlation_block <- function(size, correlation) {
    block <- matrix(correlation, nrow = size, ncol = size)
    diag(block) <- 1 # Set diagonal elements to 1
    return(block)
  }
  
  # Create each correlation block and store them in a list
  blocks <- lapply(seq_along(exposure_numbers), function(i) {
    create_correlation_block(exposure_numbers[i], correlations[i])
  })
  
  # Combine the blocks into a block diagonal matrix
  correlation_matrix <- Matrix::bdiag(blocks)
  
  # Convert to a regular matrix for compatibility
  correlation_matrix <- as.matrix(correlation_matrix)
  
  return(correlation_matrix)
}
```


### Run simulation

```{webr-r}
set.seed(1211)
df_sim <- data_gen(n_obs = 2000, n_expo = 30, n_confound = 5,
                   expo_blockNum =  c(5, 10, 15),
                   expo_blockCorr = c(0.4, 0.8, 0.1),
                   confound_blockNum = 5, confound_blockCorr = 0.2,
                   Alpha_a = Alpha_a, Alpha_c = Alpha_c, 
                   Beta_m = Beta_m, Beta_a = Beta_a,
                   Beta_c = Beta_c, Theta_c = Theta_c,
                   adjR2_M = 0.3, adjR2_Y = 0.3)
```


## Individual exposure testing (without co-exposure)

### Setup code
```{webr-r}
set.seed(1211)

expo_nm <- df_sim %>%
  select(starts_with("x")) %>%
  colnames()

confounders_nm <- df_sim %>%
  select(starts_with("c")) %>%
  colnames()

mediator_nm <- df_sim %>%
  select(starts_with("m")) %>%
  colnames()

outcome_nm <- "y"
```

### Define IndExpo_medTest_NV()
```{webr-r}
IndExpo_medTest_NV <- function(data, nboot = 1000,
                               exposures_nm, confounders_nm,
                               mediator_nm, outcome_nm) {
  suppressMessages(invisible(lapply(c("CMAverse", "purrr", "rlang"), require, character.only = T)))
  
  # extract the list of exposures
  Expo_names <- exposures_nm
  
  if (length(Expo_names) == 0) {
    stop("There are no columns with names in Expo_names")
  }
  
  # a list to store the results
  medTest_res <- list()
  
  # a function for purrr::map()
  run_cmest <- function(Expo_id, data, nboot,
                        confounders_nm = confounders_nm,
                        mediator_nm = mediator_nm,
                        outcome_nm = outcome_nm) {
    if (length(confounders_nm) == 0) {
      y_formula <- as.formula(paste0(outcome_nm, " ~ ", Expo_id, " + ", mediator_nm))
      m_formula <- as.formula(paste0(mediator_nm, " ~ ", Expo_id))
      
      # Evaluate the formulas in the correct environment
      y_model <- eval(bquote(glm(.(y_formula), family = gaussian, data = data)))
      m_model <- eval(bquote(glm(.(m_formula), family = gaussian, data = data)))
      
      # the CMA mediation effect testing
      suppressWarnings(cma_test <- CMAverse::cmest(
        data = data, model = "rb",
        full = T, EMint = F,
        yreg = y_model,
        mreg = list(m_model),
        mval = list(0),
        outcome = outcome_nm, exposure = Expo_id, mediator = mediator_nm,
        inference = "bootstrap", nboot = nboot, boot.ci.type = "per"
      ))
    } else {
      y_formula <- as.formula(paste0(
        outcome_nm, " ~ ", Expo_id, " + ", mediator_nm,
        " + ", paste0(confounders_nm, collapse = " + ")
      ))
      m_formula <- as.formula(paste0(
        mediator_nm, " ~ ", Expo_id,
        " + ", paste0(confounders_nm, collapse = " + ")
      ))
      
      # Evaluate the formulas in the correct environment
      y_model <- eval(bquote(glm(.(y_formula), family = gaussian, data = data)))
      m_model <- eval(bquote(glm(.(m_formula), family = gaussian, data = data)))
      
      # the CMA mediation effect testing
      suppressWarnings(cma_test <- CMAverse::cmest(
        data = data, model = "rb",
        full = T, EMint = F,
        yreg = y_model,
        mreg = list(m_model),
        mval = list(0), basec = c(confounders_nm),
        outcome = outcome_nm, exposure = Expo_id, mediator = mediator_nm,
        inference = "bootstrap", nboot = nboot, boot.ci.type = "per"
      ))
    }
    
    # tidy a summary table
    summary_table <- cbind(
      cma_test$effect.pe,
      cma_test$effect.se,
      cma_test$effect.ci.low,
      cma_test$effect.ci.high,
      cma_test$effect.pval
    )
    
    colnames(summary_table) <- c("Estimate", "SE", "CI_Low", "CI_Upper", "Pval")
    
    print(paste("     ", Expo_id, "Bootstrap done"))
    
    list(`CMA Test` = cma_test, `CMA Summary Table` = summary_table)
  }
  
  # run the mediation tests over all the exposures
  medTest_res <- purrr::map(Expo_names, ~ run_cmest(
    .x, data, nboot,
    confounders_nm,
    mediator_nm, outcome_nm
  ))
  names(medTest_res) <- Expo_names
  return(medTest_res)
}
```


### Run IndExpo_medTest_NV()

```{webr-r}
result <- IndExpo_medTest_NV(
  data = df_sim,
  nboot = 1000,
  exposures_nm = expo_nm,
  mediator_nm = mediator_nm,
  outcome_nm = outcome_nm,
  confounders_nm = confounders_nm
)
```


## Individual exposure testing (with co-exposure)

### Define IndExpo_medTest()

```{webr-r}
IndExpo_medTest <- function(data, nboot = 1000,
                            exposures_nm, confounders_nm,
                            mediator_nm, outcome_nm){
  
  suppressMessages(invisible(lapply(c("CMAverse", "purrr", "rlang"), require, character.only = T)))
  
  # extract the list of exposures
  Expo_names <- exposures_nm
  
  if(length(Expo_names) == 0){
    stop('There are no columns with names in Expo_names')
  }
  
  # a list to store the results
  medTest_res <- list()
  
  # a function for purrr::map()
  run_cmest <- function(Expo_id, data, nboot,
                        confounders_nm = confounders_nm,
                        mediator_nm = mediator_nm,
                        outcome_nm = outcome_nm) {
    
    if(length(confounders_nm) == 0){
      y_formula <- as.formula(paste0(outcome_nm, " ~ ", Expo_id, " + ",
                                     mediator_nm, " + ",
                                     paste0(Expo_names[Expo_names != Expo_id], collapse = " + ")))
      
      m_formula <- as.formula(paste0(mediator_nm, " ~ ", Expo_id, " + ", 
                                     paste0(Expo_names[Expo_names != Expo_id], collapse = " + ")))
      
      # Evaluate the formulas in the correct environment
      y_model <- eval(bquote(glm(.(y_formula), family = gaussian, data = data)))
      m_model <- eval(bquote(glm(.(m_formula), family = gaussian, data = data)))
      
      # the CMA mediation effect testing
      suppressWarnings(cma_test <- CMAverse::cmest(data = data, model = "rb",
                                                   full = T, EMint = F,
                                                   yreg = y_model, 
                                                   mreg = list(m_model),
                                                   mval = list(0),
                                                   basec = c(Expo_names[Expo_names != Expo_id]),
                                                   outcome = outcome_nm,
                                                   exposure = Expo_id,
                                                   mediator = mediator_nm,
                                                   inference = "bootstrap", nboot = nboot,
                                                   boot.ci.type = "per"))
    } else {
      y_formula <- as.formula(paste0(outcome_nm, " ~ ", Expo_id, " + ", mediator_nm, " + ",
                                     paste0(Expo_names[Expo_names != Expo_id], collapse = " + "),
                                     " + ", paste0(confounders_nm, collapse = " + ")))
      
      m_formula <- as.formula(paste0(mediator_nm, " ~ ", Expo_id, " + ",
                                     paste0(Expo_names[Expo_names != Expo_id], collapse = " + "),
                                     " + ", paste0(confounders_nm, collapse = " + ")))
      
      # Evaluate the formulas in the correct environment
      y_model <- eval(bquote(glm(.(y_formula), family = gaussian, data = data)))
      m_model <- eval(bquote(glm(.(m_formula), family = gaussian, data = data)))
      
      # the CMA mediation effect testing
      suppressWarnings(cma_test <- CMAverse::cmest(data = data, model = "rb",
                                                   full = T, EMint = F,
                                                   yreg = y_model, 
                                                   mreg = list(m_model),
                                                   mval = list(0),
                                                   basec = c(Expo_names[Expo_names != Expo_id], confounders_nm),
                                                   outcome = outcome_nm,
                                                   exposure = Expo_id, 
                                                   mediator = mediator_nm,
                                                   inference = "bootstrap",
                                                   nboot = nboot, boot.ci.type = "per"))
    }
    
    # tidy a summary table
    summary_table <- cbind(cma_test$effect.pe,
                           cma_test$effect.se,
                           cma_test$effect.ci.low,
                           cma_test$effect.ci.high,
                           cma_test$effect.pval)
    
    colnames(summary_table) <- c("Estimate", "SE", "CI_Low", "CI_Upper", "Pval")
    
    print(paste("     ", Expo_id, "Bootstrap done"))
    
    list(`CMA Test` = cma_test, `CMA Summary Table` = summary_table)
  }
  
  # run the mediation tests over all the exposures
  medTest_res <- purrr::map(Expo_names, ~run_cmest(.x, data, nboot,
                                                   confounders_nm,
                                                   mediator_nm, outcome_nm))
  names(medTest_res) <- Expo_names
  return(medTest_res)
}
```


### Run IndExpo_medTest()


```{webr}
set.seed(1211)
expo_nm <- df_sim %>% select(starts_with("x")) %>% colnames()
confounders_nm <- df_sim %>% select(starts_with("c")) %>% colnames()
mediator_nm <- df_sim %>% select(starts_with("m")) %>% colnames()
outcome_nm <- "y"	

result <- IndExpo_medTest(data = df_sim,
                          nboot = 1000,
                          exposures_nm = expo_nm,
                          mediator_nm = mediator_nm,
                          outcome_nm = outcome_nm,
                          confounders_nm = confounders_nm)
```